In [1]:
import pandas as pd
import sqlalchemy as sa
import urllib

params = urllib.parse.quote_plus('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=localhost;'
                      'Database=InteractDb_rc2;'
                      'Trusted_Connection=yes;')

engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
cnxn = engine.connect()

In [2]:
df_drug_ids = pd.read_sql(sa.text('SELECT $node_id, drugbank_id, atc FROM drug_nodes'), cnxn)
drug_list  = list(set(df_drug_ids['atc'].to_list()))
df_drug_ids

,$node_id_C59E823CFB2442BFB04D7FE55B42561F,drugbank_id,atc
0,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00001,B01AE02
1,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00002,L01FE01
2,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00003,R05CB13
3,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00004,L01XX29
4,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB00005,L04AB01
...,...,...,...
15230,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17382,None
15231,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17383,None
15232,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17384,None
15233,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...",DB17385,None


In [3]:
df_se_ids = pd.read_sql(sa.text('SELECT $node_id, umls_cid FROM sideeffect_nodes'), cnxn)
se_list = list(set(df_se_ids['umls_cid'].to_list()))
df_se_ids

,$node_id_E0C42641009B4A46BC30F0BA3FE81BE0,umls_cid
0,"{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",C0000727
1,"{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",C0000727
2,"{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",C0000727
3,"{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",C0000729
4,"{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",C0000729
...,...,...
75600,"{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",C3666014
75601,"{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",C3666015
75602,"{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",C3666016
75603,"{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",C3666017


In [4]:
edges_df = pd.read_csv('../parser/temp/side_effect_edges.csv', sep="\t")
edges_df

,atc,stitch_id,umls_cid,freq_ub,freq_lb
0,A16AA01,CID100000085,C0000729,0.093518,0.093518
1,A16AA01,CID100000085,C0000729,0.093518,0.093518
2,A16AA01,CID100000085,C0000737,0.093518,0.093518
3,A16AA01,CID100000085,C0000737,0.093518,0.093518
4,A16AA01,CID100000085,C0002418,0.093518,0.093518
...,...,...,...,...,...
189887,L03AB05,CID171306834,C3203358,0.166813,0.165618
189888,L03AB05,CID171306834,C3665386,0.166813,0.165618
189889,L03AB05,CID171306834,C3665386,0.166813,0.165618
189890,L03AB05,CID171306834,C3665596,0.166813,0.165618


In [5]:
edges_df = edges_df.loc[edges_df['atc'].isin(drug_list)]
edges_df = edges_df.loc[edges_df['umls_cid'].isin(se_list)]
edges_df

,atc,stitch_id,umls_cid,freq_ub,freq_lb
0,A16AA01,CID100000085,C0000729,0.093518,0.093518
1,A16AA01,CID100000085,C0000729,0.093518,0.093518
2,A16AA01,CID100000085,C0000737,0.093518,0.093518
3,A16AA01,CID100000085,C0000737,0.093518,0.093518
4,A16AA01,CID100000085,C0002418,0.093518,0.093518
...,...,...,...,...,...
189887,L03AB05,CID171306834,C3203358,0.166813,0.165618
189888,L03AB05,CID171306834,C3665386,0.166813,0.165618
189889,L03AB05,CID171306834,C3665386,0.166813,0.165618
189890,L03AB05,CID171306834,C3665596,0.166813,0.165618


In [8]:
drug_id_map = {}
for idx, row in df_drug_ids.iterrows():
    v,_,k = row.to_list()
    drug_id_map[k] = v

se_id_map = {}
for idx, row in df_se_ids.iterrows():
    v,k = row.to_list()
    se_id_map[k] = v

In [9]:
edges_df['from_id'] = [drug_id_map[i] for i in edges_df['atc'].to_list()]
edges_df['to_id'] = [se_id_map[i] for i in edges_df['umls_cid'].to_list()]
edges_df


,atc,stitch_id,umls_cid,freq_ub,freq_lb,from_id,to_id
0,A16AA01,CID100000085,C0000729,0.093518,0.093518,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff..."
1,A16AA01,CID100000085,C0000729,0.093518,0.093518,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff..."
2,A16AA01,CID100000085,C0000737,0.093518,0.093518,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff..."
3,A16AA01,CID100000085,C0000737,0.093518,0.093518,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff..."
4,A16AA01,CID100000085,C0002418,0.093518,0.093518,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff..."
...,...,...,...,...,...,...,...
189887,L03AB05,CID171306834,C3203358,0.166813,0.165618,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff..."
189888,L03AB05,CID171306834,C3665386,0.166813,0.165618,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff..."
189889,L03AB05,CID171306834,C3665386,0.166813,0.165618,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff..."
189890,L03AB05,CID171306834,C3665596,0.166813,0.165618,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff..."


In [11]:
edges_df = edges_df[['from_id','to_id','atc', 'stitch_id', 'umls_cid', 'freq_ub', 'freq_lb']]
edges_df

,from_id,to_id,atc,stitch_id,umls_cid,freq_ub,freq_lb
0,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",A16AA01,CID100000085,C0000729,0.093518,0.093518
1,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",A16AA01,CID100000085,C0000729,0.093518,0.093518
2,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",A16AA01,CID100000085,C0000737,0.093518,0.093518
3,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",A16AA01,CID100000085,C0000737,0.093518,0.093518
4,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",A16AA01,CID100000085,C0002418,0.093518,0.093518
...,...,...,...,...,...,...,...
189887,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",L03AB05,CID171306834,C3203358,0.166813,0.165618
189888,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",L03AB05,CID171306834,C3665386,0.166813,0.165618
189889,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",L03AB05,CID171306834,C3665386,0.166813,0.165618
189890,"{""type"":""node"",""schema"":""dbo"",""table"":""drug_no...","{""type"":""node"",""schema"":""dbo"",""table"":""sideeff...",L03AB05,CID171306834,C3665596,0.166813,0.165618


In [13]:
# DUMP TO DISK
edges_df.to_csv('./ingest/drug_se_edges.csv', sep="\t", index=False)